# Intro

Training notebook for Reinforcement Learning project submission.

By Ivan SVATKO, MVA 2024

### imports

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from joblib import dump, load
from gymnasium.wrappers import TimeLimit
from env_hiv import HIVPatient
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
import copy
import random

# Things that worked

Below we implement a version of Fitted Q iteration inspired by Ernst et al. 2006.

120 patiens -> 120*200 = 24000 samples

In [2]:
def collect_samples(env, total_iter, disable_tqdm=False, print_done_states=False, Q_hat=None, eps=0.15):
    """
        samples trajectories from the environment
        if provided, uses an estimated Q function with eps-greedy policy
        otherwise picks a random action

        Adapted from the sampler used in the class
    """
    s, _ = env.reset()
    #dataset = []
    S = []
    A = []
    R = []
    current_R = []
    S2 = []
    D = []
    it = 0
    cumulative_rewards = []
    for _ in tqdm(range(total_iter), disable=disable_tqdm):
        if Q_hat is None:
            a = env.action_space.sample()
        else:
            u = np.random.rand(1)
            if u>eps:
                Qsa =[]
                for a in range(env.action_space.n):
                    sa = np.append(s,a).reshape(1, -1)
                    Qsa.append(Q_hat.predict(sa))
                a = np.argmax(Qsa)
            else:
                a = env.action_space.sample()
        s2, r, done, trunc, _ = env.step(a)

        #dataset.append((s,a,r,s2,done,trunc))
        S.append(s)
        A.append(a)
        R.append(r)
        S2.append(s2)
        current_R.append(r)
        D.append(done)
        it += 1
        if done or trunc:
            cumulative_rewards.append(np.sum(current_R))
            s, _ = env.reset()
            current_R = []
            if done and print_done_states:
                print("done!")
        else:
            s = s2
    S = np.array(S)
    A = np.array(A).reshape((-1,1))
    R = np.array(R)
    S2= np.array(S2)
    D = np.array(D)
    return S, A, R, S2, D, np.mean(cumulative_rewards)

In [3]:
def et_fqi(S, A, R, S2, D, iterations, nb_actions, gamma, Q_start = None):
    """
        Adapted from the FQI loop from the class
    """
    nb_samples = S.shape[0]
    Qfunctions = Q_start
    SA = np.append(S,A,axis=1)
    for it in tqdm(range(iterations)):
        if Qfunctions is None:
             value=R.copy()
        else:
            Q2 = np.zeros((nb_samples,nb_actions))
            for a2 in range(nb_actions):
                A2 = a2*np.ones((S.shape[0],1))
                S2A2 = np.append(S2,A2,axis=1)
                Q2[:,a2] = Qfunctions.predict(S2A2)
            max_Q2 = np.max(Q2,axis=1)
            value = R + gamma*(1-D)*max_Q2
        Q = HistGradientBoostingRegressor()
        Q.fit(SA,value)
        Qfunctions = Q
    return Qfunctions

### Stages loop

In [6]:
N_stages = 20
n_patients = 20
shuffle_stages = [3, 5, 7, 9, 11, 13, 15]
Q_functions = [None]
episode_length = 200
fqi_iter = 100
gamma = 0.98

env = TimeLimit(
             env=HIVPatient(domain_randomization=False), max_episode_steps=episode_length
            )

# -We either start from scratch or load a dataset an a model-
# 0- stage

print("Collecting the first sample")
S, A, R, S2, D, cum_rew = collect_samples(env, n_patients*episode_length, Q_hat=Q_functions[-1])
sample = [S,A,R,S2,D]
_s = dump(sample, f"samples\sample_{0}")
print(f"Stage: {0} \t strategy: {Q_functions[-1]}_{0} \t reward: {cum_rew:e}")
print("Fitting the Q function")

Q_next = et_fqi(S, A, R, S2, D, fqi_iter, 4, gamma, Q_start = Q_functions[-1])
Q_functions.append(Q_next)
_s = dump(Q_next, f"samples\Q{0}")


# ------------------loading data and models------------------

# use saved data
#sample_saved = load("samples\Run_20p_200it_100upd\sample_19")
#S, A, R, S2, D = sample_saved

# generate new data
#S, A, R, S2, D, cum_rew = collect_samples(env, n_patients*episode_length, Q_hat=Q_functions[-1])

#Q_functions.append(load("samples\Run_20p_200it_100upd\Q19"))

# number of pretrained epochs
lag = 0

for n in range(1, N_stages):
    if n in shuffle_stages:
        env = TimeLimit(
             env=HIVPatient(domain_randomization=True), max_episode_steps=episode_length
            )
    else:
        env = TimeLimit(
             env=HIVPatient(domain_randomization=False), max_episode_steps=episode_length
            )
    print(f"Sampling with: \t {Q_functions[-1]}")
    S_next,A_next,R_next,S2_next,D_next, cum_rew = collect_samples(env, n_patients*episode_length, Q_hat=Q_functions[-1])
    print(f"Stage: {n+lag} \t strategy: {Q_functions[-1]}_{n+lag} \t reward: {cum_rew:e}")
    S = np.vstack([S, S_next])
    A = np.vstack([A, A_next])
    R = np.hstack([R, R_next])
    S2 = np.vstack([S2, S2_next])
    D = np.hstack([D, D_next])
    sample = [S,A,R,S2,D]
        
    _s = dump(sample, f"samples\sample_{n+lag}")

    print(f"Fitting the Q function, sample size: \t {S.shape[0]}")

    Q_next = et_fqi(S, A, R, S2, D, fqi_iter, 4, gamma, Q_start = Q_functions[-1])
    _s = dump(Q_next, f"samples\Q{n+lag}")
    Q_functions.append(Q_next)

100%|██████████| 4000/4000 [02:38<00:00, 25.31it/s]


Stage: 0 	 strategy: None_0 	 reward: 1.041681e+07
Fitting the Q function


100%|██████████| 100/100 [00:40<00:00,  2.50it/s]


Sampling with: 	 HistGradientBoostingRegressor()


100%|██████████| 4000/4000 [03:10<00:00, 20.96it/s]


Stage: 1 	 strategy: HistGradientBoostingRegressor()_1 	 reward: 4.904563e+09
Fitting the Q function, sample size: 	 8000


  5%|▌         | 5/100 [00:02<00:45,  2.08it/s]


KeyboardInterrupt: 

# Things that didn't work

Additionally, several versions of DQN were attempted. Notably target networks were explored. Using EMA weight update resulted in the most stable traing performace. The code below reuses the same code, modifying it to implement a clipped double DQN.

These attempts were ultimately unsucsessful.

In [ ]:


env = TimeLimit(
    env=HIVPatient(domain_randomization=False), max_episode_steps=200
) 
state_dim = env.observation_space.shape[0]
n_action = env.action_space.n 
nb_neurons=128
DQN1 = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, n_action))

DQN2 = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, nb_neurons),
                          nn.ReLU(),
                          nn.Linear(nb_neurons, n_action))

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.data = []
        self.index = 0 
    def append(self, s, a, r, s_, d):
        if len(self.data) < self.capacity:
            self.data.append(None)
        self.data[self.index] = (s, a, r, s_, d)
        self.index = int((self.index + 1) % self.capacity)
    def sample(self, batch_size):
        batch = random.sample(self.data, batch_size)
        return list(map(lambda x:torch.Tensor(np.array(x)), list(zip(*batch))))
    def __len__(self):
        return len(self.data)
    

class ProjectAgent:
    def __init__(self, config, model_1, model_2):
        self.batch_size = config['batch_size']
        self.nb_actions = config['nb_actions']
        self.memory = ReplayBuffer(config['buffer_size'])
        self.epsilon_max = config['epsilon_max']
        self.epsilon_min = config['epsilon_min']
        self.epsilon_stop = config['epsilon_decay_period']
        self.epsilon_delay = config['epsilon_delay_decay']
        self.warmup_steps = config['warmup_steps']
        #self.update_target_strategy = config['update_target_strategy']# if 'update_target_strategy' in config.keys() else 'replace'
        #self.update_target_freq = config['update_target_freq']# if 'update_target_freq' in config.keys() else 20
        #self.update_target_tau = config['update_target_tau']# if 'update_target_tau' in config.keys() else 0.005
        self.epsilon_step = (self.epsilon_max-self.epsilon_min)/self.epsilon_stop
        self.gamma = 0.98
        self.model_1 = model_1 
        self.model_2 = model_2
        self.criterion = torch.nn.MSELoss()
        self.optimizer_1 = torch.optim.Adam(self.model_1.parameters(), lr=config['learning_rate'])
        self.optimizer_2 = torch.optim.Adam(self.model_2.parameters(), lr=config['learning_rate'])
    
    def gradient_step(self):
        if len(self.memory) > self.batch_size:
            self.optimizer_1.zero_grad()
            self.optimizer_2.zero_grad()
            X, A, R, Y, D = self.memory.sample(self.batch_size)
            #QYmax = self.model(Y).max().detach().item()
            QYmax_1, id = self.model_1(Y).max(1)
       
            QYmax_1 = QYmax_1.detach()

            QYmax_2 = self.model_2(Y).detach()
       
            QYmax_2 = QYmax_2.gather(1, id.view(-1, 1)).detach()
   
            QYmin = torch.min(QYmax_1, QYmax_2)
            #update = torch.addcmul(R, self.gamma, 1-D, QYmin).detach()
            update = torch.addcmul(R, 1-D, QYmin, value=self.gamma)
            QXA_1 = self.model_1(X).gather(1, A.to(torch.long).unsqueeze(1))
            QXA_2 = self.model_2(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss_1 = self.criterion(QXA_1, update.unsqueeze(1))
            loss_2 = self.criterion(QXA_2, update.unsqueeze(1))
            loss_1.backward()
            loss_2.backward()
            self.optimizer_1.step()
            self.optimizer_2.step() 

    def train(self, env, max_episode):
        episode_return = []
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0

        for _s in range(self.warmup_steps):
            action = self.act(state, use_random=True)
            next_state, reward, done, trunc, _ = env.step(action)
            self.memory.append(state, action, reward, next_state, trunc)
        print(f"size of warmup buffer is: {len(self.memory)}")

        while episode < max_episode:

            if step > self.epsilon_delay:
                epsilon = max(self.epsilon_min, epsilon-self.epsilon_step)


            if np.random.rand() < epsilon:
                action = self.act(state, use_random=True)
            else:

                action = self.act(state)
            next_state, reward, done, trunc, _ = env.step(action)

            self.memory.append(state, action, reward, next_state, trunc)
            episode_cum_reward += reward

            # train
            self.gradient_step()
            # if self.update_target_strategy == 'replace':
            #     if step % self.update_target_freq == 0: 
            #         self.target_model.load_state_dict(self.model.state_dict())
            # if self.update_target_strategy == 'ema':
            #     target_state_dict = self.target_model.state_dict()
            #     model_state_dict = self.model.state_dict()
            #     tau = self.update_target_tau
            #     for key in model_state_dict:
            #         target_state_dict[key] = tau*model_state_dict[key] + (1-tau)*target_state_dict[key]
            #     self.target_model.load_state_dict(target_state_dict)

            # next transition
            step += 1
            if done or trunc:
                episode += 1
                print("Episode ", '{:3d}'.format(episode), 
                      ", epsilon ", '{:6.2f}'.format(epsilon), 
                      ", buffer length ", '{:5d}'.format(len(self.memory)), 
                      #", episode return ", '{:4.1f}'.format(episode_cum_reward),
                      ", episode return ", '{:e}'.format(episode_cum_reward),
                      sep='')
                state, _ = env.reset()
                episode_return.append(episode_cum_reward)
                episode_cum_reward = 0
            else:
                state = next_state

        return episode_return

    def act(self, observation, use_random=False):
        if use_random:
            return env.action_space.sample()
        return torch.argmax(self.model_1(torch.FloatTensor(observation)).unsqueeze(0)).item()

    def save(self, path):
        torch.save(self.model_1.state_dict(), "best1.pt")
        torch.save(self.model_2.state_dict(), "best2.pt")
    def load(self):
        self.model_1.load_state_dict(torch.load("best1.pt"))
        self.model_2.load_state_dict(torch.load("best2.pt"))



In [ ]:
config = {'nb_actions': env.action_space.n,
          'learning_rate': 0.0001,
          'gamma': 0.98,
          'buffer_size': 1e6,
          'epsilon_min': 0.05,
          'epsilon_max': 0.5,
          'epsilon_decay_period': 10000,
          'epsilon_delay_decay': 600,
          #'update_target_strategy': 'ema',
          #'update_target_freq': 50,
          #'update_target_tau': 0.005,
          'batch_size': 1024,
          'warmup_steps': 2000}


agent = ProjectAgent(config, DQN1, DQN2)
scores = agent.train(env, 200)
plt.plot(scores)